In [1]:
import torch
from torch import nn
from model import TransformerModel
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
train_data_path = "../../train_daily.csv"
test_data_path = "../../test_daily.csv"

In [3]:
input_hours = 90       # 输入序列长度（90天）
output_hours = 90 

use_cols = [
    'Global_active_power','Global_reactive_power',
    'Global_intensity','Sub_metering_1',
    'Sub_metering_2','Sub_metering_3',
    'sub_metering_remainder'
]

def load_hourly(path):
    df = pd.read_csv(path, parse_dates=['DateTime'])
    df.sort_values('DateTime', inplace=True)
    df.reset_index(drop=True, inplace=True)

    # 计算 sub_metering_remainder
    df["sub_metering_remainder"] = (
        df["Global_active_power"] * 1000 / 60
        - df["Sub_metering_1"]
        - df["Sub_metering_2"]
        - df["Sub_metering_3"]
    ).clip(lower=0)

    df = df[['DateTime'] + use_cols].copy()
    df.ffill(inplace=True)
    df.bfill(inplace=True)
    return df

train_df = load_hourly(train_data_path)
test_df  = load_hourly(test_data_path) 


full_df = pd.concat([train_df, test_df], ignore_index=True)

# ========= 标准化 ========= #
# scaler = MinMaxScaler()
# scaled = scaler.fit_transform(full_df[use_cols])
# target = scaled[:, 0]  # 目标值：Global_active_power
scaled = full_df[use_cols].values  # 直接使用原始值
target = scaled[:, 0]


# ========= 滑动窗口构造函数 ========= #
def build_samples(data, target, input_len, output_len, step=1):
    X, y = [], []
    for i in range(input_len, len(data) - output_len + 1, step):
        X.append(data[i - input_len:i])
        y.append(target[i:i + output_len])
    return np.array(X), np.array(y)

# ========= 构建样本 ========= #
X_all, y_all = build_samples(scaled, target, input_hours, output_hours, step=1)


# ========= 拆分 train / test ========= #
split_index = len(train_df) - input_hours  # 同样逻辑保证 test 在真实 test 部分
X_train = X_all[:split_index]
y_train = y_all[:split_index]
X_test  = X_all[split_index:]
y_test  = y_all[split_index:]


print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test  shape: {X_test.shape}")
print(f"y_test  shape: {y_test.shape}")

X_train shape: (657, 90, 7)
y_train shape: (657, 90)
X_test  shape: (606, 90, 7)
y_test  shape: (606, 90)


In [4]:
from torch.utils.data import TensorDataset, DataLoader, random_split
import matplotlib.pyplot as plt
import os

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
batch_size = 16

train_ds = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.float32)
)
val_size = int(len(train_ds) * 0.2)
train_size = len(train_ds) - val_size
train_subset, val_subset = random_split(train_ds, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

In [7]:
print(test_df['Global_active_power'].describe())


count    695.000000
mean       1.061744
std        0.356459
min        0.146000
25%        0.837435
50%        1.053375
75%        1.272331
max        2.304042
Name: Global_active_power, dtype: float64


In [5]:
model = TransformerModel(
    input_dim=7,
    d_model=256,
    nhead=8,
    num_layers=3,
    output_seq_len=90,
    dropout=0.3,
    input_seq_len=90  # 这是你实际的序列长度
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

train_losses, val_losses = [], []
best_val = float('inf')
best_model = None
patience = 10
no_improve = 0

for epoch in range(1, 31):
    model.train()
    total_train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        tgt = torch.zeros_like(xb).to(device)

        optimizer.zero_grad()
        pred = model(xb, src_mask=None)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * xb.size(0)
    avg_train = total_train_loss / len(train_subset)
    train_losses.append(avg_train)

    # 验证
    all_preds, all_gts = [], []
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            tgt = torch.zeros_like(xb).to(device)
            pred = model(xb, src_mask=None)
            loss = criterion(pred, yb)
            total_val_loss += loss.item() * xb.size(0)
            all_preds.append(pred.cpu().numpy())
            all_gts.append(yb.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_gts = np.concatenate(all_gts, axis=0)

    avg_val = total_val_loss / len(val_subset)
    val_losses.append(avg_val)

    val_mae = mean_absolute_error(all_gts, all_preds)
    val_mse = mean_squared_error(all_gts, all_preds)
    print(f"Epoch {epoch:03d} | Train Loss: {avg_train:.6f} | Val Loss: {avg_val:.6f} | MAE: {val_mae:.4f} | MSE: {val_mse:.4f}")

    if avg_val < best_val:
        best_val = avg_val
        best_model = model.state_dict()
        torch.save(best_model, "test.pth")
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

plt.figure(figsize=(8, 5))
plt.plot(train_losses, label="Train MSE")
plt.plot(val_losses, label="Val MSE")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.title("Loss Curve")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("test.png", dpi=300)
plt.close()

Epoch 001 | Train Loss: 0.425024 | Val Loss: 0.236442 | MAE: 0.3839 | MSE: 0.2364
Epoch 002 | Train Loss: 0.258781 | Val Loss: 0.211223 | MAE: 0.3632 | MSE: 0.2112
Epoch 003 | Train Loss: 0.234768 | Val Loss: 0.179613 | MAE: 0.3310 | MSE: 0.1796
Epoch 004 | Train Loss: 0.219529 | Val Loss: 0.168216 | MAE: 0.3194 | MSE: 0.1682
Epoch 005 | Train Loss: 0.216282 | Val Loss: 0.164963 | MAE: 0.3163 | MSE: 0.1650
Epoch 006 | Train Loss: 0.203664 | Val Loss: 0.167700 | MAE: 0.3215 | MSE: 0.1677
Epoch 007 | Train Loss: 0.193128 | Val Loss: 0.146187 | MAE: 0.2965 | MSE: 0.1462
Epoch 008 | Train Loss: 0.189454 | Val Loss: 0.145649 | MAE: 0.2949 | MSE: 0.1456
Epoch 009 | Train Loss: 0.185903 | Val Loss: 0.143337 | MAE: 0.2937 | MSE: 0.1433
Epoch 010 | Train Loss: 0.173585 | Val Loss: 0.125254 | MAE: 0.2715 | MSE: 0.1253
Epoch 011 | Train Loss: 0.162600 | Val Loss: 0.130540 | MAE: 0.2807 | MSE: 0.1305
Epoch 012 | Train Loss: 0.156154 | Val Loss: 0.130481 | MAE: 0.2819 | MSE: 0.1305
Epoch 013 | Trai

In [5]:
import torch
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# 1️⃣ 加载模型
model = TransformerModel(
    input_dim=7,
    d_model=256,
    nhead=8,
    num_layers=3,
    output_seq_len=90,
    dropout=0.3,
    input_seq_len=90
).to(device)

model.load_state_dict(torch.load("test.pth", map_location=device))
model.eval()

# 2️⃣ 构造测试集 DataLoader
test_ds = torch.utils.data.TensorDataset(
    torch.tensor(X_test, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.float32)
)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=128, shuffle=False)

# 3️⃣ 推理
all_preds = []
all_gts = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        preds = model(xb).cpu().numpy()
        all_preds.append(preds)
        all_gts.append(yb.numpy())

all_preds = np.concatenate(all_preds, axis=0)  # [样本数, 90]
all_gts = np.concatenate(all_gts, axis=0)      # [样本数, 90]

# 4️⃣ 评估指标
mse = mean_squared_error(all_gts.flatten(), all_preds.flatten())
mae = mean_absolute_error(all_gts.flatten(), all_preds.flatten())
print(f"Test MAE: {mae:.4f}")
print(f"Test MSE: {mse:.4f}")

# 5️⃣ 绘图示例
for i in range(5):
    plt.figure(figsize=(10, 4))
    plt.plot(all_gts[i], label="Ground Truth")
    plt.plot(all_preds[i], label="Prediction")
    plt.xlabel("Days into the Future")
    plt.ylabel("Daily Global Active Power (Sum)")
    plt.title(f"Transformer: 90-Day Power Consumption Prediction")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{i}.png", dpi=300)
    plt.close()


/tmp/ipykernel_861515/2426192477.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("test.pth", map_location=device))


Test MAE: 0.4215
Test MSE: 0.2730
